In [1]:
import polars as pl
from transformers import AutoTokenizer

import warnings
import os
os.chdir('/home/denisalpino/dev/FinABYSS')

In [2]:
warnings.simplefilter("ignore", Warning)

In [3]:
title_lower = pl.col("title").str.to_lowercase()
condition_title = (
    # PR Newswire group
    (pl.col("source") == "PR Newswire") & (
        title_lower.str.contains("releases monthly portfolio", literal=True) |
        title_lower.str.contains("announce distribution rates", literal=True)
    )
) | (
    # Business Wire group
    (pl.col("source") == "Business Wire") & (
        title_lower.str.starts_with("eurofins: purchases of own shares") |
        title_lower.str.starts_with("eurofins scientific") |
        title_lower.str.starts_with("notice of public") |
        title_lower.str.ends_with("short interest report") |
        title_lower.str.ends_with("home member state election") |
        title_lower.str.contains("advisory for", literal=True) |
        title_lower.str.contains("voting rights", literal=True) |
        title_lower.str.contains("early repurchase", literal=True) |
        title_lower.str.contains("early redemption", literal=True) |
        title_lower.str.contains("issue of debt", literal=True) |
        title_lower.str.contains("variable rate fix", literal=True) |
        title_lower.str.contains("voting rights", literal=True) |
        title_lower.str.contains("disclosure of transactions", literal=True) |
        title_lower.str.contains("securit", literal=True)
    )
) | (
    # Josh Jenke group
    (pl.col("source") == "Josh Jenke, Peoria Journal Star") &
        title_lower.str.starts_with("3 of the best")
) | (
    # CNW News group
    (pl.col("source") == "CNW News") &
        title_lower.str.contains("evening lottery winning numbers", literal=True)
) | (
    # ACCESS Newswire group
    (pl.col("source") == "ACCESS Newswire") & (
        title_lower.str.contains("corporation earnings call", literal=True) |
        title_lower.str.starts_with("3 of the best")
    )
) | (
    # ETF Trends group
    (pl.col("source") == "ETF Trends") &
        title_lower.str.starts_with("main management market note")
) | (
    # Public Record group
    (pl.col("source") == "Public Record") &
        title_lower.str.contains("vendors licenses", literal=True)
) | (
    # Investing.com group
    (pl.col("source") == "Investing.com") &
        title_lower.str.starts_with("watch live:")
) | (
    # GlobeNewswire group
    (pl.col("source") == "GlobeNewswire") & (
        title_lower.str.starts_with('key digital') |
        title_lower.str.starts_with("issue of equity") |
        title_lower.str.starts_with("delisting of certificates") |
        title_lower.str.starts_with("fixing of coupon rates") |
        title_lower.str.ends_with("presentation") |
        title_lower.str.contains("conditions? for") |
        title_lower.str.contains("buy-?back program") |
        title_lower.str.contains("disclosure of trading in own shares", literal=True) |
        title_lower.str.contains("announces tax treatment", literal=True) |
        title_lower.str.contains("annual general meeting information", literal=True) |
        title_lower.str.contains("voting rights", literal=True) |
        title_lower.str.contains("notification of major holdings", literal=True) |
        title_lower.str.contains("recall", literal=True) |
        title_lower.str.contains("public health alert", literal=True) |
        title_lower.str.contains("annual account", literal=True) |
        title_lower.str.contains("calendar", literal=True) |
        title_lower.str.contains("announcement analyst targets", literal=True) |
        title_lower.str.contains("release of the half year", literal=True) |
        title_lower.str.contains("in own shares", literal=True) |
        title_lower.str.contains("declaration", literal=True) |
        title_lower.str.contains("monthly volume summary", literal=True) |
        title_lower.str.contains("corporation announces", literal=True) |
        title_lower.str.contains("disclosure of transactions", literal=True) |
        title_lower.str.contains("auction announcement", literal=True) |
        title_lower.str.contains("mandatory notification of trade", literal=True) |
        title_lower.str.contains("auction announcement", literal=True) |
        title_lower.str.contains("notification of major shareholding", literal=True) |
        title_lower.str.contains(r"managers[\S\s]+transactions") |
        title_lower.str.contains(r"\b(gm|agm)\b")
    )
) | (
    # Wooster Daily Record group
    (pl.col("source") == "Wooster Daily Record") &
        title_lower.str.contains("annual financial report", literal=True)
) | (
    # K-12 Dive group
    (pl.col("source") == "K-12 Dive") &
        title_lower.str.contains("test yourself on this week", literal=True)
) | (
    # etf.com group
    (pl.col("source") == "etf.com") &
        title_lower.str.starts_with("top 10")
) | (
    # General condition group
    title_lower.str.starts_with('aem united states ag tractor and combine report') |
    title_lower.str.starts_with("null") |
    title_lower.str.contains("net asset value", literal=True) |
    title_lower.str.contains("media advisory", literal=True) |
    title_lower.str.contains("form 8", literal=True) |
    title_lower.str.contains("block listing interim review", literal=True) |
    title_lower.str.contains("holdings in company", literal=True) |
    title_lower.str.contains("holding(s) in company", literal=True) |
    title_lower.str.contains("director/pdmr shareholding", literal=True) |
    title_lower.str.contains("director / pdmr notification", literal=True) |
    title_lower.str.contains("results of operations", literal=True)
)

In [4]:
text_lower = pl.col("text").str.to_lowercase()
condition_text = (
    # GlobeNewswire group
    (pl.col("source") == "GlobeNewswire") & (
        text_lower.str.starts_with('attachment') |
        text_lower.str.starts_with('as announced') |
        text_lower.str.starts_with('company announcement') |
        text_lower.str.starts_with('dear madam, sir, please find hereunder the links') |
        text_lower.str.starts_with('form 8')
    )
) | (
    # Zacks group
    (pl.col("source") == "Zacks") & (
        text_lower.str.starts_with('want the latest recommendations from zacks')
    )
) | (
    # Sarah Ferguson group
    (pl.col("source") == "Sarah Ferguson") &
        text_lower.str.starts_with("(sponsored)")
) | (
    # Exec Edge group
    (pl.col("source") == "Exec Edge") &
        text_lower.str.starts_with("pe edge and exec edge are pleased")
) | (
    # CoinDesk group
    (pl.col("source") == "CoinDesk") &
        text_lower.str.starts_with("<a href=")
) | (
    # General condition group
    text_lower.str.starts_with("(repeat") |
    text_lower.str.starts_with("on episode") |
    text_lower.str.starts_with("On today’s episode")
)

In [5]:
if not os.path.exists("data/preprocessed/"):
    os.mkdir("data/preprocessed/")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

In [7]:
# TODO: Improve tokenization filtering by optimization of usage `map_elements()` (mb `map_batches()` would be beter)

# Takes nearly 1 hour due to non-matrix non-parallelizable tokenization operation
with pl.Config() as cfg:
    cfg.set_streaming_chunk_size(25_000)

    # Build lazy plan
    plan = (
        pl.scan_parquet("data/raw/articles.parquet")
        .with_columns(
            pl.col("text")
            # Delete all spaces from sides
            .str.strip_chars()
            # Compress whitespace characters with a line break into only one line break
            .str.replace_all(r"\n\s+", "\n")
            .str.replace_all(r"\s+\n", "\n")
            # Compress multiple whitespace characters into a single character
            .str.replace_all(u"\xa0", " ")
            .str.replace_all(r"[^\S\n]{2,}", " ")
            .alias("text")
        )
        # Filter empty texts
        .filter(pl.col("text").str.len_chars() > 0)
        # Filter non-representative texts by titles
        .filter(~condition_title)
        # Filter non-representative texts by texts
        .filter(~condition_text)
        # Cast the column to its corresponding type
        .with_columns(
            pl.col("datetime")
              .str.to_datetime(format="%Y-%m-%dT%H:%M:%S%.fZ", time_zone="UTC")
              .alias("datetime")
        )
        # Sorting by datetime asceding
        .sort("datetime")
        # During process of deduplication we cant compare items
        # by column `assets` due to internal library restrictions
        .unique(subset=["text"], keep="first")
        # Filter by quantity of tokens
        #.filter(
        #    pl.col("text")
        #    .map_elements(
        #        function=lambda text: len(tokenizer.tokenize(text)),
        #        return_dtype=pl.UInt32
        #    ) < 8196
        #)
        .sink_parquet("data/preprocessed/articles.parquet", engine="streaming")
    )

In [8]:
actual_size = os.path.getsize("data/preprocessed/articles.parquet")

print(f"Real size on disk: {actual_size / 1024**3:.2f} GB")

Real size on disk: 2.12 GB
